In [59]:
import quantfreedom as qf
import numpy as np
import pandas as pd
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go
import talib
from talib.abstract import Function

from quantfreedom.backtester.indicators.talib_ind import from_talib, talib_func_list_website_link, talib_ind_info
from quantfreedom.backtester.evaluators.evaluators import eval_is_below, eval_is_above, combine_evals
from quantfreedom._typing import (
    pdFrame, pdSeries
)
prices = pd.read_csv('E:/Coding/backtesters/QuantFreedom/tests/data/30min.csv', index_col='time')
rsi_ind = from_talib(
    func_name='rsi',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=30,
)


In [11]:
# Define Divergence
def divergence(
    # using price set as starting point to calculate divergence
    price_set,
    # using last 14 bars/days period window for RSI indicator. This is just a template and can be changed when we will execute .run function in next stage.
    rsi_window,
    # using last 1 bar/day period window for fast Moving Average indicator. This is just a template and can be changed when we will execute .run function in next stage.
    fast_ma_window,

    # using last 50 bars/days period window for fast Moving Average indicator.  This is just a template and can be changed when we will execute .run function in next stage.
        slow_ma_window,):

    slow_ma = from_talib(
        func_name='MA',
        df_prices=prices,
        cart_product=False,
        combos=False,
        timeperiod=slow_ma_window,
    )
    fast_ma = from_talib(
        func_name='MA',
        df_prices=prices,
        cart_product=False,
        combos=False,
        timeperiod=fast_ma_window,
    )
    rsi_indicator = from_talib(
        func_name='MA',
        df_prices=prices,
        cart_product=False,
        combos=False,
        timeperiod=rsi_window,
    ).squeeze()

# Calculate the Divergence indicator. Arithmetics behind indicators is black box for me and consist of pure mathematics
    bullish_divergence = ((price_set.diff() < 0) & (rsi_indicator.diff() > 0)).to_numpy()

    bearish_divergence = ((price_set.diff() < 0) &
                          (rsi_indicator.diff() < 0)).to_numpy()

    # Determine UpTrend (fast MA above slow MA)
    trend_up = eval_is_above(
        ind_data=fast_ma,
        ind_results=slow_ma.squeeze(),
    )
    trend_down = eval_is_below(
        ind_data=fast_ma,
        ind_results=slow_ma.squeeze(),
    )

    # Convert everything to Numpy array
    trend_up = trend_up.to_numpy().flatten()
    trend_down = trend_down.to_numpy().flatten()

    # Combine all previous indicators into the one
    # We need that if bullish_divergence (trend_up), True PLUS fast MA above slow MA, then set 1, everything else - 0. If bearish_divergence True, PLUS fast MA above slow MA (trend_down), then set -1
    indicator_buy = np.logical_and(bullish_divergence == True,trend_up == True)
    indicator_buy = np.where(indicator_buy == True, 1, 0)
    indicator_sell = np.logical_and(bearish_divergence == True,trend_down == True)
    indicator = np.where(indicator_sell == True, -1, indicator_buy)


    return indicator

atr_ind = from_talib(
    func_name='atr',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=14,
)

talib.SMA(atr_ind.values.flatten(), timeperiod=50)



In [12]:
divergence(
    price_set=prices.close,
    rsi_window=15,
    fast_ma_window=10,
    slow_ma_window=40,
)

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
atr_ind = from_talib(
    func_name='atr',
    df_prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=14,
)

In [15]:
atr_ind

ATR_timeperiod,14
time,
2020-10-01 00:00:00,NaN
2020-10-01 00:30:00,NaN
2020-10-01 01:00:00,NaN
2020-10-01 01:30:00,NaN
2020-10-01 02:00:00,NaN
...,...
2021-02-22 21:30:00,960.749362
2021-02-22 22:00:00,937.588693
2021-02-22 22:30:00,909.225215


In [25]:
talib.SMA(atr_ind.values.flatten(), timeperiod=50)


array([         nan,          nan,          nan, ..., 751.24061144,
       760.87985348, 770.26772109])

In [52]:
(1000 / prices.close.values.flatten())

array([0.09267841, 0.0927902 , 0.09265265, ..., 0.01849215, 0.01867449,
       0.01850447])

In [58]:
talib.SMA(atr_ind.values.flatten(), timeperiod=50) * (1000 / prices.close.values.flatten())

array([        nan,         nan,         nan, ..., 13.89205413,
       14.20903945, 14.25339504])

In [ ]:
fast_ma = vbt.MA.run(price_set, fast_ma_window, short_name='fast')
#printing tpe of bearish_divergence 
print("printing tpe of bearish_divergence dataset:")
print(type(bearish_divergence))
# Get slow Moving Average (MA) of the prices

atr = vbt.ATR.run(high_price, low_price, price, atr_window)
#atr = atr.atr.to_numpy()

atr = atr.atr.to_numpy()
##print(type(atr))

# Calculate adaptive/dynamically changing value based on ATR and current price 
atr_results = atr * atr_scaler / price #* rsi_window

slow_ma = vbt.MA.run(atr_results, slow_ma_window, short_name='slow')
print("printing tpe of slow_ma dataset:")
print(type(slow_ma))

In [49]:
fig = go.Figure()
fig.add_scatter(
    x=atr_ind.index.to_list(),
    y=prices.close.values.flatten(),
    mode='lines',
    marker=dict(color='red'),
    name='ATR'

)
fig.add_scatter(
    x=rsi_ind.index.to_list(),
    y=talib.SMA(atr_ind.values.flatten(), timeperiod=100) * 1000 / prices.close.values.flatten(),
    mode='lines',
    marker=dict(color='blue'),
    name='SMA'
)
fig.show()


In [66]:
atr_ind.values.flatten()

array([         nan,          nan,          nan, ..., 909.22521497,
       911.60198533, 898.84470066])

In [69]:
atr_ind

ATR_timeperiod,14
time,
2020-10-01 00:00:00,NaN
2020-10-01 00:30:00,NaN
2020-10-01 01:00:00,NaN
2020-10-01 01:30:00,NaN
2020-10-01 02:00:00,NaN
...,...
2021-02-22 21:30:00,960.749362
2021-02-22 22:00:00,937.588693
2021-02-22 22:30:00,909.225215


In [71]:
atr_ma = from_talib(
    func_name='MA',
    df_prices=None,
    cart_product=False,
    combos=False,
    timeperiod=50,
    price=atr_ind.values.flatten()
)

ValueError: price must be a string and not a list of strings

In [32]:
talib_ind_info('stoch')

{'name': 'STOCH',
 'group': 'Momentum Indicators',
 'display_name': 'Stochastic',
 'function_flags': None,
 'input_names': OrderedDict([('prices', ['high', 'low', 'close'])]),
 'parameters': OrderedDict([('fastk_period', 5),
              ('slowk_period', 3),
              ('slowk_matype', 0),
              ('slowd_period', 3),
              ('slowd_matype', 0)]),
 'output_flags': OrderedDict([('slowk', ['Dashed Line']),
              ('slowd', ['Dashed Line'])]),
 'output_names': ['slowk', 'slowd']}

In [35]:
close = np.random.random(100)
open = np.random.random(100)
high = np.random.random(100)

In [61]:
Function('sma')(np.random.random(10000), 50)

array([       nan,        nan,        nan, ..., 0.40719212, 0.41578604,
       0.42300347])

In [47]:
_, hey = talib.STOCH(close, open, high, *(10,15))
hey

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
       52.20622177, 52.66668398, 51.6478533 , 50.56555857, 50.47024434,
       49.2819345 , 47.84125667, 45.57349477, 46.39699963, 49.00099783,
       53.44319656, 56.49473102, 58.02738735, 57.20362031, 54.87816286,
       53.6363963 , 53.16044162, 53.74532525, 53.63715409, 52.50142721,
       51.33084818, 51.3060523 , 52.89918675, 54.88815407, 55.99779457,
       55.99447097, 57.45398368, 58.10845452, 60.14077332, 61.36564124,
       61.63228781, 60.68903848, 57.6953449 , 55.85781135, 53.68601926,
       54.77236938, 55.25824177, 54.9882949 , 51.96765861, 49.07580278,
       47.48500152, 44.45895558, 42.94830214, 38.84190794, 39.83